In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd '/content/drive/My Drive/pcanet/PCANet_python_ver/'

/content/drive/My Drive/pcanet/PCANet_python_ver


In [0]:
# ! git pull

Already up to date.


In [0]:
ls

colab_CIFAR10.ipynb  data_loader.py  PCANet-MNIST.ipynb  __pycache__/
data/                interface.py    PCANet.py           README.md
data_loader2.py      LICENSE         playground.ipynb


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.autograd import Variable
import os

In [0]:
torch.cuda.is_available()

True

In [0]:
transform_train = transforms.Compose([
    transforms.Resize((224,224), interpolation=2),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.Resize((224,224), interpolation=2),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [0]:
vgg11 = models.vgg11_bn(pretrained=True)
vgg11.classifier._modules['6'] = nn.Linear(4096, 10, bias=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg11.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
vgg11 = torch.nn.DataParallel(vgg11).cuda()

Downloading: "https://download.pytorch.org/models/vgg11_bn-6002323d.pth" to /root/.cache/torch/checkpoints/vgg11_bn-6002323d.pth


In [0]:
vgg11

DataParallel(
  (module): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU(inplace=True)
      (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [0]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    vgg11.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inps, tars) in enumerate(trainloader):
        inputs = Variable(inps.cuda())
        targets = Variable(tars.cuda())
        # inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = vgg11(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                    % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [0]:
for i in range(1,11):
    train(i)


Epoch: 1
Loss: 0.449 | Acc: 84.690% (42345/50000)

Epoch: 2
Loss: 0.201 | Acc: 93.134% (46567/50000)

Epoch: 3
Loss: 0.122 | Acc: 95.804% (47902/50000)

Epoch: 4
Loss: 0.084 | Acc: 97.058% (48529/50000)

Epoch: 5
Loss: 0.068 | Acc: 97.596% (48798/50000)

Epoch: 6
Loss: 0.054 | Acc: 98.108% (49054/50000)

Epoch: 7
Loss: 0.048 | Acc: 98.386% (49193/50000)

Epoch: 8
Loss: 0.035 | Acc: 98.858% (49429/50000)

Epoch: 9
Loss: 0.036 | Acc: 98.770% (49385/50000)

Epoch: 10
Loss: 0.031 | Acc: 99.002% (49501/50000)


In [0]:
def test():
    
    vgg11.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inps, tars) in enumerate(testloader):

            inputs = Variable(inps.cuda())
            targets = Variable(tars.cuda())
            outputs = vgg11(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    
    print('Saving..')
    state = {
        'net': vgg11.state_dict(),
        'acc': acc,
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/ckpt.pth')
    best_acc = acc


In [0]:
test()

Loss: 0.261 | Acc: 92.670% (9267/10000)
Saving..
